In [0]:
df_clean = spark.read.parquet("/Volumes/workspace/default/projects/cleaned_data/mouse_viral_study_cleaned.parquet")

df_clean = df_clean.withColumn("virus_present", df_clean["virus_present"].cast("boolean"))

# display(df_clean)

df_pd = df_clean.toPandas()

In [0]:
import mlflow


In [0]:
CATALOG_NAME = "workspace"
SCHEMA_NAME = "default"

In [0]:
mvis_table = f"{CATALOG_NAME}.{SCHEMA_NAME}.mvis"
spark.sql(f"DROP TABLE IF EXISTS {mvis_table}")
df_clean.write.format("delta").saveAsTable(mvis_table)

In [0]:
import numpy as np
import pandas as pd
import sklearn.datasets
import sklearn.metrics
import sklearn.model_selection
import sklearn.ensemble
import sklearn.svm

import matplotlib.pyplot as plt

from hyperopt import fmin, tpe, hp, SparkTrials, Trials, STATUS_OK
from hyperopt.pyll import scope

In [0]:
data_labels = df_pd['virus_present']
df_pd = df_pd.drop(['virus_present'], axis=1)

In [0]:
# Split 80/20 train-test
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
  df_pd,
  data_labels,
  test_size=0.2,
  random_state=1
)

In [0]:
mlflow.autolog()

2025/07/05 13:44:54 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/07/05 13:44:54 WARNING mlflow.spark: With Pyspark >= 3.2, PYSPARK_PIN_THREAD environment variable must be set to false for Spark datasource autologging to work.
2025/07/05 13:44:54 WARNING mlflow.tracking.fluent: Exception raised while enabling autologging for pyspark: [JVM_ATTRIBUTE_NOT_SUPPORTED] Attribute `sparkContext` is not supported in Spark Connect as it depends on the JVM. If you need to use this attribute, do not use Spark Connect when creating your session. Visit https://spark.apache.org/docs/latest/sql-getting-started.html#starting-point-sparksession for creating regular Spark Session in detail.
2025/07/05 13:44:54 WARNING mlflow.tracking.fluent: Exception raised while enabling autologging for pyspark.ml: [JVM_ATTRIBUTE_NOT_SUPPORTED] Attribute `sparkContext` is not supported in Spark Connect as it depends on the JVM. If you need to use this attribute, do not use Spark Connec

In [0]:
import mlflow
import mlflow.sklearn
import sklearn
from sklearn import svm, metrics
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

import databricks.connect as db_connect
import mlflow.tracking._model_registry.utils

mlflow.tracking._model_registry.utils._get_registry_uri_from_spark_session = lambda: "databricks-uc"

mlflow.login() 
spark_ctx = db_connect.DatabricksSession.builder.serverless(True).getOrCreate()

search_space = {
    'C': hp.loguniform('C', -3, 3),
    'kernel': hp.choice('kernel', ['linear', 'rbf', 'poly']),
    'gamma': hp.choice('gamma', ['scale', 'auto'])
}

def train_model(params):
    mlflow.sklearn.autolog()
    
    with mlflow.start_run(nested=True):
        model = svm.SVC(
            C=params['C'],
            kernel=params['kernel'],
            gamma=params['gamma'],
            probability=True,
            random_state=0
        )
        
        model.fit(X_train, y_train)
        
        predicted_probs = model.predict_proba(X_test)
        auc = sklearn.metrics.roc_auc_score(y_test, predicted_probs[:, 1])
        
        mlflow.log_metric('test_auc', auc)
        mlflow.sklearn.log_model(model, "model")
        
        return {'status': STATUS_OK, 'loss': -1 * auc, 'model':model}

trials = Trials()

with mlflow.start_run(run_name='svm_hyperopt') as main_run:
    best_params = fmin(
        fn=train_model,
        space=search_space,
        algo=tpe.suggest,
        max_evals=32,
        trials=trials
    )
    
    print("Best hyperparameters found:", best_params)

2025/07/05 13:44:56 INFO mlflow.utils.credentials: Successfully connected to MLflow hosted tracking server! Host: https://dbc-499a9713-b50f.cloud.databricks.com.
/databricks/python/lib/python3.11/site-packages/databricks/connect/session.py:391: UserWarning: Ignoring the default notebook Spark session and creating a new Spark Connect session. To use the default notebook Spark session, use DatabricksSession.builder.getOrCreate() with no additional parameters.
  warnings.warn(new_notebook_session_msg)


  0%|          | 0/32 [00:00<?, ?trial/s, best loss=?]

Uploading artifacts:   0%|          | 0/3 [00:00<?, ?it/s]

Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

2025/07/05 13:45:13 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.11.4/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.



...

Uploading artifacts:   0%|          | 0/3 [00:00<?, ?it/s]

Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

2025/07/05 13:49:04 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.11.4/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.



Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

100%|██████████| 32/32 [04:08<00:00,  7.75s/trial, best loss: -1.0]
Best hyperparameters found: {'C': 0.19525541873031102, 'gamma': 1, 'kernel': 0}


In [0]:
best_run = mlflow.search_runs(
  order_by=['metrics.test_auc DESC', 'start_time DESC'],
  max_results=10,
).iloc[0]
print('Best Run')
print('AUC: {}'.format(best_run["metrics.test_auc"]))
print('Run ID : {}'.format(best_run["run_id"]))

Best Run
AUC: 1.0
Run ID : 8b0e74a62db6487d8978a18e59377c1d


In [0]:
import mlflow
from mlflow.models.signature import infer_signature
import mlflow.sklearn

model_uri = 'runs:/8b0e74a62db6487d8978a18e59377c1d/model'

# Load the model
model = mlflow.sklearn.load_model(model_uri)

if mlflow.active_run() is not None:
    mlflow.end_run()

# Start a new run
with mlflow.start_run() as run:
    # (Optional) Retrain if needed
    # model.fit(X_train, y_train)

    # Infer signature
    signature = infer_signature(X_train, model.predict(X_train))

    # Log model with signature
    mlflow.sklearn.log_model(model, "model", signature=signature)

    # New URI for registration
    new_model_uri = f"runs:/{run.info.run_id}/model"




Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

Registered model 'workspace.default.mvis_model_m' already exists. Creating a new version of this model...


Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

---------------------------------------------------------------------------
MlflowException                           Traceback (most recent call last)
File <command-6879499967690092>, line 28
     25     new_model_uri = f"runs:/{run.info.run_id}/model"
     27 # Register using new URI
---> 28 mlflow.register_model(new_model_uri, f"{CATALOG_NAME}.{SCHEMA_NAME}.mvis_model_m")

File /databricks/python/lib/python3.11/site-packages/mlflow/tracking/_model_registry/fluent.py:77, in register_model(model_uri, name, await_registration_for, tags)
     17 def register_model(
     18     model_uri,
     19     name,
   (...)
     22     tags: Optional[Dict[str, Any]] = None,
     23 ) -> ModelVersion:
     24     """Create a new model version in model registry for the model files specified by ``model_uri``.
     25 
     26     Note that this method assumes the model registry backend URI is the same as that of the
   (...)
     75         Version: 1
     76     """
---> 77     return _register_mod

In [0]:
import mlflow

mlflow.artifacts.download_artifacts('runs:/8b0e74a62db6487d8978a18e59377c1d/model', dst_path="./svm")


/databricks/python/lib/python3.11/site-packages/mlflow/protos/service_pb2.py:11: UserWarning: google.protobuf.service module is deprecated. RPC implementations should provide code generator plugins which generate code specific to the RPC implementation. service.py will be removed in Jan 2025
  from google.protobuf import service as _service


'/Workspace/Users/dhinaz2025@gmail.com/Mouse Viral Infection Study/svm/model'